# **ETH Price and Users Behavior on Compound**

- The main purpose of this notebok ois to first present a summary of statistics on and see if there are sEit correlations between e market pricece of ETH and borrowers behaviours
- **Another idea to pursue is the assumption that people may take short and long positions depending on the ETH price trend (increase-bullish or deceaeaea-bshri)**  
- consider the amount of cETH-cUSDC and cUSDC-cETH pairs.
- Plot whether the Ethereum price trend (up or down) may influence people to take more long or short stratiessx)

Importing the necessary modules

In [116]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Preparing the data & creating a df for ETH

In [ ]:
csv_file_path = r'../data/daily_balance_sheet.csv'
df = pd.read_csv(csv_file_path)
liability = pd.read_csv(r'../data/daily_liability_matrix.csv')
col_drop = ['repayUSD','bufferUSD', 'haircutUSD', 'seizeUSD']
df.drop(col_drop, axis = 1)

csv_file_path2 = r'../data/daily_markets.csv'
markets = pd.read_csv(csv_file_path2)
markets['date'] = pd.to_datetime(markets['date'])
df['date'] = pd.to_datetime(df['date'])
df_eth = df[df['symbol'] == 'cETH']


Plotting total borrow, total collateral and total assets

In [ ]:
chart1 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_line().encode(
    x='date:T',
    y='totalBorrowUSD:Q',
    color=alt.value('blue'),
    tooltip=['date', 'totalBorrowUSD']
).properties(
    width=600,
    height=300
)
chart2 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_line().encode(
    x='date:T',
    y = 'totalCollateralUSD:Q',
    tooltip=['date:T', 'totalCollateralUSD:Q'],
    color=alt.value('orange')
).properties(
    width=600,
    height=300
)
chart3 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_line().encode(
    x='date:T',
    y = 'totalAssetsUSD:Q',
    tooltip=['date:T', 'totalAssetsUSD:Q'],
    color=alt.value('red')
).properties(
    width=600,
    height=300
)

(chart1 +chart2+chart3)#.resolve_scale(y='independent')



Plotting total assets for most used coins 

In [ ]:
#see https://messari.io/report/state-of-compound-q4-2021 for most used coin
most_used_coins = ['cETH', 'cUSDC', 'cBTC', 'cDAI', 'cUSDT']

df_copy = df
df_copy = df_copy[df_copy['symbol'].isin(most_used_coins)]

chart1 = alt.Chart(df_copy).mark_area(
    interpolate='linear',
    line=True
).encode(
    x='date:T',
    y=alt.Y('totalAssetsUSD:Q',stack = None, title='Total Assets USD'),
    color='symbol:N',
    tooltip=['date:T', 'totalAssetsUSD:Q', 'symbol:N']
).properties(
    title='Total Assets USD Over Time',
    width=600,
    height=300
)


chart1


Total borrow and total assets for ETH

In [ ]:

chart1 = alt.Chart(df_eth).mark_line().encode(
    x = 'date:T',
    y = 'totalBorrowUSD:Q',
    tooltip=['date:T', 'totalLiabilitiesUSD:Q'],
    color=alt.value('blue')
)

chart2 = alt.Chart(df_eth).mark_line().encode(
    x = 'date:T',
    y = 'totalAssetsUSD:Q',
    tooltip=['date:T', 'totalCollateralUSD:Q'],
    color=alt.value('orange')
)

(chart1 + chart2).resolve_scale(y='independent') | (chart1+chart2).properties(title = 'Total borrow and assets in cETH, assets in orange')



Total assets and liabilities in USDC

In [ ]:
chart1 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_area().encode(
    x='date:T',
    y=alt.Y('totalAssetsUSD:Q'),
    color=alt.Color('variable:N', scale=alt.Scale(scheme='category10')),
    tooltip=['date:T', 'value:Q'],
).transform_fold(
    ['tokensUSD', 'reservesUSD', 'interAssetsUSD'],
    as_=['variable', 'value']
).properties(
    title='Total assets in USDC',
    width=600,
    height=400
)

chart2 = alt.Chart(df[df['symbol'] == 'cUSDC']).mark_area().encode(
    x='date:T',
    y=alt.Y('totalLiabilitiesUSD:Q'),
    color=alt.Color('variable:N', scale=alt.Scale(scheme='category10')),
    tooltip=['date:T', 'value:Q'],
).transform_fold(
    ['bufferUSD', 'depositsUSD', 'interLiabilitiesUSD', 'netWorthUSD'],
    as_=['variable', 'value']
).properties(
    title='Total liabilities in USDC',
    width=400,
    height=400
)
chart1|chart2


Reserves of coins over time

In [ ]:
most_used_coins = ['cETH', 'cUSDC', 'cBTC', 'cDAI', 'cUSDT']

chart = alt.Chart(df_copy).mark_line().encode(
    x='date:T',
    y='reservesUSD:Q',
    color='symbol:N',
    tooltip=['date:T', 'reservesUSD:Q', 'symbol:N']
).properties(
    title='Reserves for All Coins Over Time',
    width=600,
    height=300
)
chart




Plotting ETH price and checking correlation of col USDC- bor ETH and vice-versa
- First I have to create a dataset for ETH market (because 'underlyingPriceUSD' is not in 'Liability' but in 'daily_markets'
- Then I filter, first I only choose ETH as collateral in 'liability' then for the same dataset I filter for only borrowing USDC

In [ ]:
#Creating df for ETH
ETH_markets = markets.loc[markets['symbol'] == 'cETH']

ETH_USD = liability[liability['symbol_col'] == 'cETH']
ETH_USD = ETH_USD[ETH_USD['symbol_bor'] == 'cUSDC']

ETH_USD['date'] = pd.to_datetime(ETH_USD['date']) #to merge with ETH_markets 
ETH_USD = pd.merge(ETH_USD, ETH_markets[['date', 'underlyingPriceUSD', 'borrowRate', 'supplyRate']], on='date')

chart1 = alt.Chart(ETH_USD).mark_line().encode(
    x = 'date:T',
    y = 'underlyingPriceUSD:Q',
    tooltip=['date:T', 'underlyingPriceUSD:Q'],
    color=alt.value('orange')
).properties(
    width=600,
    height=300
).interactive()

chart2 = alt.Chart(ETH_USD).mark_line().encode(
    x = 'date:T',
    y = 'matchBorrowUSD:Q',
    tooltip=['date:T', 'matchBorrowUSD:Q'],
    color=alt.value('black')
).properties(
    width=600,
    height=300
).interactive()
(chart1 +chart2).properties(title = "Price of ETH (orange), and quantity of USDC borrowed with ETH(black)").resolve_scale(y='independent')


"""We do same but this time, use USD to borrow ETH"""
USD_ETH = liability[liability['symbol_col'] == 'cUSDC']
USD_ETH = USD_ETH[USD_ETH['symbol_bor'] == 'cETH']
USD_ETH['date'] = pd.to_datetime(USD_ETH['date']) #to merge with ETH_markets 
USD_ETH= pd.merge(USD_ETH, ETH_markets[['date', 'underlyingPriceUSD', 'borrowRate', 'supplyRate']], on='date')
chart3 = alt.Chart(USD_ETH).mark_line().encode(
    x='date:T',
    y='matchBorrowUSD:Q',
    tooltip=['date:T', 'matchBorrowUSD:Q'],
    color=alt.value('black')
).properties(
    width=600,
    height=300
).interactive()
(chart1 +chart2).properties(title = "Price of ETH (orange), and quantity of USDC borrowed with ETH(black)").resolve_scale(y='independent')|(chart1 +chart3).properties(title = "Price of ETH (orange), and quantity of ETH borrowed with USDC(black)").resolve_scale(y='independent')



**Regressing collateral on price of ETH : here I use liability df to test if the price goes up, do we use more and more ETH for borrowing USDc (that would be long position)?**

In [ ]:

#Regressing on borrow rate, supply rate and price of ETH
ETH_USD['const'] = 1
y = ETH_USD['matchCollateralUSD']
X = ETH_USD.loc[:, ['underlyingPriceUSD', 'borrowRate', 'supplyRate']]

X_scaled = StandardScaler().fit_transform(X)


model = sm.OLS(y, X_scaled).fit(cov_type='HC3')
print(model.summary())

**Regressing USD-ETH borrow on price of ETH : here I use liability df to test if the price goes up, do we use more and more USDC for borrowing ETH (that would be short position)?**

In [ ]:

#Regressing on borrow rate, supply rate and price of ETH
USD_ETH['const'] = 1
y = USD_ETH['matchBorrowUSD']
X = USD_ETH[['underlyingPriceUSD', 'borrowRate', 'supplyRate']]

X_scaled = StandardScaler().fit_transform(X)


model = sm.OLS(y, X_scaled).fit(cov_type='HC3')
print(model.summary())